# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [5]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import Geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [29]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
#vacation_df.head(10)
vacation_df.loc[vacation_df["Country"]=="BR"].head(10)


,City_ID,City,Country,Max Temp,Current Weather,Lat,Hotel Name,Lng
4,27,Touros,BR,78.55,overcast clouds,-5.1989,Pousada Atlântico,-35.4608
8,64,Sao Borja,BR,79.95,clear sky,-28.6606,Hotel Obino,-56.0044
19,115,Cabedelo,BR,79.47,few clouds,-6.9811,No hotel found,-34.8339
22,130,Itarema,BR,79.20,overcast clouds,-2.9248,Pousada Oásis,-39.9167
25,146,Barao De Melgaco,BR,81.55,overcast clouds,-16.1944,No hotel found,-55.9675
31,190,Itaqui,BR,87.08,clear sky,-29.1253,Hotel Contursi,-56.5531
35,219,Tutoia,BR,80.20,overcast clouds,-2.7619,Pousada San Vicente,-42.2744
60,418,Guiratinga,BR,79.95,overcast clouds,-16.3494,No hotel found,-53.7583
62,443,Rio Branco,BR,79.41,scattered clouds,-9.9747,No hotel found,-67.8100
72,530,Bela Cruz,BR,78.10,overcast clouds,-3.0506,No hotel found,-40.1678


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [8]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country","Current Description"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [30]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"]=="Touros"]
vacation_end = vacation_df.loc[vacation_df["City"]=="Sao Borja"]
vacation_stop1 = vacation_df.loc[vacation_df["City"]=="Cabedelo"]
vacation_stop2 = vacation_df.loc[vacation_df["City"]=="Itarema"]
vacation_stop3 = vacation_df.loc[vacation_df["City"]=="Itaqui"]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [31]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start,vacation_stop1])
itinerary_df = pd.concat([itinerary_df, vacation_stop2])
itinerary_df = pd.concat([itinerary_df, vacation_stop3])
itinerary_df = pd.concat([itinerary_df, vacation_end])

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Current Weather,Lat,Hotel Name,Lng
4,27,Touros,BR,78.55,overcast clouds,-5.1989,Pousada Atlântico,-35.4608
19,115,Cabedelo,BR,79.47,few clouds,-6.9811,No hotel found,-34.8339
22,130,Itarema,BR,79.20,overcast clouds,-2.9248,Pousada Oásis,-39.9167
31,190,Itaqui,BR,87.08,clear sky,-29.1253,Hotel Contursi,-56.5531
8,64,Sao Borja,BR,79.95,clear sky,-28.6606,Hotel Obino,-56.0044


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [32]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lat", "Lng"]]

# Display sample data
waypoints_df

,Lat,Lng
4,-5.1989,-35.4608
19,-6.9811,-34.8339
22,-2.9248,-39.9167
31,-29.1253,-56.5531
8,-28.6606,-56.0044


## Use GeoViews to create map that shows the four cities in the itinerary

In [33]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "City",
    hover_cols = ["Country","Current Weather"]
)

In [34]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Current Weather)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [35]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": Geoapify_key,
}

In [36]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '53a1f5c7b1b3421a98076f3c526b05d3',
 'waypoints': '-5.1989,-35.4608|-6.9811,-34.8339|-2.9248,-39.9167|-29.1253,-56.5531|-28.6606,-56.0044'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [37]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [-35.4608, -5.1989], 'original_index': 0},
     {'location': [-34.8339, -6.9811], 'original_index': 1},
     {'location': [-39.9167, -2.9248], 'original_index': 2},
     {'location': [-56.5531, -29.1253], 'original_index': 3},
     {'location': [-56.0044, -28.6606], 'original_index': 4}],
    'units': 'metric',
    'distance': 5652749,
    'distance_units': 'meters',
    'time': 234853.28,
    'legs': [{'distance': 292054,
      'time': 12542.72,
      'steps': [{'from_index': 0,
        'to_index': 13,
        'distance': 203,
        'time': 9.852,
        'instruction': {'text': 'Drive northwest on Avenida Prefeito José Américo.'}},
       {'from_index': 13,
        'to_index': 32,
        'distance': 686,
        'time': 61.85,
        'instruction': {'text': 'Bear right onto Avenida 27 de Março.'}},
       {'from_index': 32,
        'to_index': 91,
        'distance': 3705,
        

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [52]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]
legs

{'type': 'MultiLineString',
 'coordinates': [[[-35.460721, -5.198762],
   [-35.460922, -5.198647],
   [-35.460977, -5.198622],
   [-35.461041, -5.198601],
   [-35.461098, -5.19859],
   [-35.461309, -5.198565],
   [-35.4615, -5.198552],
   [-35.461833, -5.198535],
   [-35.461986, -5.19851],
   [-35.462141, -5.198481],
   [-35.462247, -5.198459],
   [-35.46231, -5.198442],
   [-35.462375, -5.198422],
   [-35.462496, -5.198374],
   [-35.462886, -5.19814],
   [-35.463338, -5.197871],
   [-35.463775, -5.197613],
   [-35.464145, -5.197381],
   [-35.464263, -5.197321],
   [-35.46429, -5.197307],
   [-35.464913, -5.196989],
   [-35.465084, -5.196909],
   [-35.465318, -5.196808],
   [-35.465568, -5.196714],
   [-35.46575, -5.196649],
   [-35.465754, -5.196648],
   [-35.466444, -5.196394],
   [-35.466973, -5.196183],
   [-35.467313, -5.196047],
   [-35.467754, -5.195876],
   [-35.467974, -5.195782],
   [-35.468011, -5.195765],
   [-35.468065, -5.195757],
   [-35.468274, -5.195622],
   [-35.46856

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [55]:
# Create and empty list to store the longitude of each step
lng = []

# Create and empty list to store the latitude of step
lat = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in legs["coordinates"][0]:
   lat.append(leg[0])
   lng.append(leg[1])
    

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [56]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame({
    "lat":lat,
    "lng":lng
})

# Add the steps' longitude and latitude from each step as columns to the DataFrame
# YOUR CODE HERE

# Display sample data
route_df

,lat,lng
0,-35.460721,-5.198762
1,-35.460922,-5.198647
2,-35.460977,-5.198622
3,-35.461041,-5.198601
4,-35.461098,-5.198590
...,...,...
3382,-34.831432,-6.980884
3383,-34.832026,-6.980961
3384,-34.832886,-6.981075
3385,-34.833499,-6.981146


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [58]:
# Configure the route path by using the GeoViews' Path function
route_path = route_df.hvplot.points(
    "lng",
    "lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
)


In [59]:
# Display a composed plot by using the route_map and route_path objects
route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lng,lat]